# RGB-D Fusion

This exercise will describe how to fuse images from known poses into one pointcloud.
The excercise is divided into the following steps.

1. Load recorded images and poses.
2. 

Note: if the interactive viewer does not work you may have to restart the notebook with `%matplotlib widget` instead of `%matplotlib notebook`

Note: install jupyter in your conda environment via `conda install -c conda-forge notebook` and the other missing packages. Start it in the command line via` jupter notebook --port xxxx`. Use the vs-code pop up to open jupyter in your local server.

Note: we use [Open3D](http://www.open3d.org/) for this exercise. Unfortunately the visualization requires OpenGL, which fails for setups like ours, where the visualization and the code run on separate machines. A minimal working solution (with drawbacks in quality) is to start a jupyter notebook on one of the pool machines (not the login) and use vscode for port-forwarding it to your local browser.
**The recommended way is to run everything on your local machine**

In [ ]:
# %matplotlib notebook
# %matplotlib widget
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
try:
    import open3d as o3d
except ImportError:
    pass

log = logging.getLogger(__name__)
np.set_printoptions(suppress=True)

In [ ]:
# Download the data required for this exercise (30Mb)
! wget https://lmb.informatik.uni-freiburg.de/people/argusm/wd_40.tar
! tar -xvf wd_40.tar
# or find them here
! ls /project/cv-ws2122/shared-data1/wd_40

# 1. Load Recorded Images

This cell defines a data class that loads images and data from files.

Please complete the `get_projection_matrix` function.

In [ ]:
import json
from PIL import Image

class ViewLoader:
    def __init__(self, base_path):
        self.base_path = base_path
        assert os.path.isdir(base_path)
        files = sorted(os.listdir(self.base_path))
        files = [f for f in files if (f.startswith("rgb_") and f.endswith(".png"))]
        self.max_idx = int(files[-1].replace("rgb_", "").replace(".png", ""))
        print(f"Loaded {self.max_idx+1} images.")
        
    def __len__(self):
        return self.max_idx + 1
    
    def __getitem__(self, idx):
        return self.get_rgbdp(idx)

    def get_info(self):
        info = {
            "camera": {"calibration": 
            {"width": 640, "height": 480,
            "fx": 617.8902587890625, "fy": 617.8903198242188,
            "ppx": 315.20367431640625, "ppy": 245.70614624023438}}
        }
        return info
    
    def get_intrinsics(self):
        info = self.get_info()
        calib = info["camera"]["calibration"]
        return calib
    
    def get_K(self):
        calib = self.get_intrinsics()
        cam_intrinsic = np.eye(3)
        cam_intrinsic[0, 0] = calib["fx"]
        cam_intrinsic[1, 1] = calib["fy"]
        cam_intrinsic[0, 2] = calib["ppx"]
        cam_intrinsic[1, 2] = calib["ppy"]
        return cam_intrinsic
    
    def get_robot_pose(self, idx, return_dict=False):
        pose_file = os.path.join(self.base_path, "pose_{0:04d}.json".format(idx) )
        with open(pose_file,"rb") as f_obj:
            pose = json.load(f_obj)
        pose_m = np.eye(4)
        pose_m[:3, :3] = R.from_euler("xyz", [pose[x] for x in ['rot_x', 'rot_y', 'rot_z']]).as_matrix()
        pose_m[:3, 3] = [pose[x] for x in ['x', 'y', 'z']]
        if return_dict:
            return pose_m, pose
        else:
            return pose_m
    
    def get_rgb_file(self, idx):
        rgb_file = os.path.join(self.base_path, "rgb_{0:04d}.png".format(idx) )
        return rgb_file
    
    def get_depth_file(self, idx):
        depth_file = os.path.join(self.base_path, "depth_{0:04d}.png".format(idx) )
        return depth_file    
    
    def get_rgbdp(self, idx):
        rgb_file = self.get_rgb_file(idx)
        depth_file = self.get_depth_file(idx)
        
        pose_m, pose_d = self.get_robot_pose(idx, True)
        # depth
        depth_scaling = pose_d["depth_scaling"]
        rgb  = np.asarray(Image.open(rgb_file))
        depth = np.asarray(Image.open(depth_file), dtype=np.float32) * depth_scaling
        return rgb, depth, pose_m
        
    def get_cam_pose(self, idx, marker_dir="pose_marker_one"):
        marker_dir = os.path.join(self.base_path, marker_dir)
        fn  = "{0:08d}.json".format(idx)
        pose_fn = os.path.join(marker_dir, fn)
        with open(pose_fn, "r") as fo:
            T = np.array(json.load(fo))
        return T
    
    def get_projection_matrix(self):
        # returns a 4x3 projection matrix using the intrinsics
        intr = self.get_intrinsics()
        cam_mat = np.array([[intr['fx'], 0, intr['ppx'], 0],
                            [0, intr['fy'], intr['ppy'], 0],
                            [0, 0, 1, 0]])
        assert cam_mat.shape == (3, 4)
        return cam_mat
    

    def project(self, X):
        """
        Project an (homogenous) cartesian coordinate into the camera frame.
        """
        if X.shape[0] == 3:
            if len(X.shape) == 1:
                X = np.append(X, 1)
            else:
                X = np.concatenate([X, np.ones((1, X.shape[1]))], axis=0)

        x = self.get_projection_matrix() @ X
        result = np.round(x[0:2] / x[2]).astype(int)
        width, height = self.get_intrinsics()['width'], self.get_intrinsics()['height']
        if not (0 <= result[0] < width and 0 <= result[1] < height):
            log.warning("Projected point outside of image bounds")
        return result[0], result[1]

# vl = ViewLoader(base_path="TODO:insert path to wd_40 here")
vl = ViewLoader(base_path="wd_40")
print("camera calibration:")
camera_calibration = vl.get_K()
K = np.array(camera_calibration)
print(K.round(2))

In [ ]:
from ipywidgets import widgets, Layout, interact

fig, ax = plt.subplots(1)
image, depth, pose = vl.get_rgbdp(2)
line = ax.imshow(np.asarray(image))
ax.set_axis_off()

def update(w):
    image, depth, pose = vl.get_rgbdp(w)
    line.set_data(np.asarray(image))
    fig.canvas.draw_idle()
    # plt.imshow(np.asarray(image))
    # plt.show()
    
slider_w = widgets.IntSlider(min=2, max=len(vl)-1, step=1, value=0,
                             layout=Layout(width='70%'))
interact(update, w=slider_w)

# 2. Show Marker Detection Results.

To simplify things marker detection has already been run. Next we want to verify its results.
Do this by completing the `get_projection_matrix` function in the ViewLoader.
Then draw a coordinate frame into each image for which we have detection results.
The coordinate frame should have axis lengths of 10cm, with x=red, y=green, and z=blue.
This can be done using `PIL.ImageDraw`.

In [ ]:
from PIL import ImageDraw

def show_marker_pose(image, T_cam_object):
    """
    draw the coordinate frame into each image for which we have detection results
    Arguments:
        image: image as numpy.ndarray
        T_cam_object: transform from object into cam x_cam = T_cam_object @ x
    Returns:
        im: image (should be PIL.Image.Image)
    """
    print(type(image))
    print(T_cam_object.shape)
    # TODO

    #  using PIL.ImageDraw
    
    center = np.array([0, 0, 0, 1])
    x = np.array([0.1, 0, 0, 1])
    y = np.array([0, 0.1, 0, 1])
    z = np.array([0, 0, 0.1, 1])

    object_cam = T_cam_object @ center
    x_cam = T_cam_object @ x
    y_cam = T_cam_object @ y
    z_cam = T_cam_object @ z

    object_cam_p = vl.project(object_cam)
    x_cam_p = vl.project(x_cam)
    y_cam_p = vl.project(y_cam)
    z_cam_p = vl.project(z_cam)

    im = Image.fromarray(image)
    draw = ImageDraw.Draw(im)
    draw.line(object_cam_p + x_cam_p, fill="red", width=5)
    draw.line(object_cam_p + y_cam_p, fill="green", width=5)
    draw.line(object_cam_p + z_cam_p, fill="blue", width=5)

    # end TODO
    
    # type(im) should be PIL.Image.Image
    return im

image, depth, robot_pose = vl.get_rgbdp(2)
T_cam_object = vl.get_cam_pose(2)

fig, ax = plt.subplots(1)
image_m = show_marker_pose(image, T_cam_object)
line = ax.imshow(np.asarray(image_m))
ax.set_axis_off()

def update(w):
    image, depth, pose = vl.get_rgbdp(w)
    try:
        T_cam_object = vl.get_cam_pose(w)
    except FileNotFoundError:
        print("No pose estimation.")
        line.set_data(np.asarray(image))
        return
    image_m = show_marker_pose(image, T_cam_object)
    line.set_data(np.asarray(image_m))
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=2, max=len(vl)-1, step=1, value=0,
                             layout=Layout(width='70%'))
interact(update, w=slider_w)

In [ ]:
def get_tcp_marker_lists(m_dir="pose_marker_one"):
    T_robot_tcp_list = []
    T_cam_marker_list = []
    for i in range(len(vl)):
        try:
            robot_pose = vl.get_robot_pose(i)
            cam_pose = vl.get_cam_pose(i, marker_dir=m_dir)
        except (FileNotFoundError, ValueError):
            continue
        T_robot_tcp_list.append(robot_pose)
        T_cam_marker_list.append(cam_pose)

    return np.array(T_robot_tcp_list), np.array(T_cam_marker_list)

plot_o3d = True
# plot_o3d = False
if plot_o3d:
    T_robot_tcp_list, T_cam_marker_list = get_tcp_marker_lists()
    mesh_frames = []
    for T_robot_tcp, T_cam_marker in zip(T_robot_tcp_list, T_cam_marker_list):
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
        mesh_frame.transform(np.linalg.inv(T_robot_tcp))
        mesh_frames.append(mesh_frame)
        
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.20)
        mesh_frame.transform(T_cam_marker)
        mesh_frames.append(mesh_frame)

    o3d.visualization.draw_geometries(mesh_frames)

# Display Merged Pointclouds

In [ ]:
import open3d as o3d

first_rgb = Image.open(vl.get_rgb_file(2))
K_o3d = o3d.camera.PinholeCameraIntrinsic()
K_o3d.set_intrinsics(first_rgb.size[1], first_rgb.size[0],
                     K[0, 0], K[1, 1], K[0, 2], K[1, 2])

pcd_list = []
for i in range(2,len(vl)):
    try:
        rgb_file = Image.open(vl.get_rgb_file(i))
        depth_file = Image.open(vl.get_depth_file(i))
        T_c = vl.get_cam_pose(i)
        T_r = vl.get_robot_pose(i)
        depth_scaling = vl.get_robot_pose(i, return_dict=True)[1]["depth_scaling"]
    except (FileNotFoundError, ValueError):
        continue
    
    rgb = o3d.geometry.Image(np.array(rgb_file))
    depth = o3d.geometry.Image(np.array(depth_file).astype(np.uint16))
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb, depth,
                                          depth_scale=1.0/depth_scaling, depth_trunc=1.0,
                                          convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, K_o3d)
    
    #T_est = T_r @ T_calib
    T_est = np.linalg.inv(T_c)
    pcd.transform(T_est)
    pcd_list.append(pcd)

# sum pointclouds for easier visualization
pcd_all = pcd_list[0]
for pcd_cur in pcd_list[1:]:
    pcd_all += pcd_cur
o3d.visualization.draw_geometries([pcd_all])

# TODO: improve the pointcloud
- remove the gripper
- unify the points on the surface
- extract a mesh

Just try out many things. You can find some inspiration in [tutorials](http://www.open3d.org/docs/0.8.0/index.html#tutorial-index). Make sure the docs version and your installed version match, since some functionality has changed.